<a href="https://colab.research.google.com/github/mirali24/Automation-Python/blob/main/Excel_Report_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Automating an Excel Report with Python**

In [ ]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font

#Function for automating the spreadsheet
def automate_report(inputFilename, outputFilename):
    df = pd.read_excel(inputFilename)

    # Add a new column for Total Revenue
    df['Total Sales'] = df['Unit Price'] * df['Order Quantity']
    df['Total Cost'] = df['Unit Cost'] * df['Order Quantity']
    df['Revenue'] = df['Total Sales'] - df['Total Cost']

    # Create charts
    fig_bar = px.bar(df, x='Sales Channel', y='Revenue', title='Revenue Per Channel',color='Sales Channel')
    fig_bar.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='lightgray',
    shapes=[dict(
        type='rect',
        xref='paper',
        yref='paper',
        x0=0,
        y0=0,
        x1=1,
        y1=1,
        line=dict(color='black', width=2))],
    width=500,
    height=500
)



    # Create a pie chart for Sales Channel distribution
    fig_pie = px.pie(df, names='Sales Channel', title='Sales Channel Distribution')

    # Customize layout
    fig_pie.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='lightgray',
        width=500,
        height=500)




    # Save the graphs as image files
    pio.write_image(fig_bar, '/content/drive/MyDrive/automation_in_python/bar_graph.png')
    pio.write_image(fig_pie, '/content/drive/MyDrive/automation_in_python/pie_chart.png')
    writer = pd.ExcelWriter(outputFilename, engine='xlsxwriter')
    df.to_excel(writer, index=False, sheet_name='Channel Data')
    worksheet = writer.sheets['Channel Data']
    worksheet.set_column('F:F', 12)
    worksheet.insert_image('L1', '/content/drive/MyDrive/automation_in_python/bar_graph.png')
    worksheet.insert_image('L30','/content/drive/MyDrive/automation_in_python/pie_chart.png')
    # save the file
    writer.close()

#Function for automating the reports
def automate_formatting(inputFilename, outputFilename):
    # # load the Excel file
    wb = load_workbook(inputFilename)
    ws = wb.active

    # # define the fill pattern to highlight the column
    fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')

    # highlight the 'Product' column of the sheet
    for cell in ws['A:A']:
        cell.fill = fill

    # =========================Borders======================
    # set a border around a specific range of cells
    range_border = Border(left=Side(style='medium'),
                        right=Side(style='medium'),
                        top=Side(style='medium'),
                        bottom=Side(style='medium'))

    for row in ws.iter_rows(min_row=1, max_row=7992, min_col=1, max_col=ws.max_column):
        for cell in row:
            cell.border = range_border

    # =====================column alignmnet====================
    # # set the width of all columns to 20
    for col in ws.columns:
        ws.column_dimensions[col[0].column_letter].width = 20

    # add some text to the worksheet and set it left-aligned
    for row in ws.iter_rows(min_row=1, max_row=7992, min_col=1, max_col=ws.max_column):
        for cell in row:
            # cell.value = f'Text {cell.column}{cell.row}'
            cell.alignment = Alignment(horizontal='left')

    #=======================font styles===============
    # set font styles for different rows
    font_name = Font(name='Times New Roman', bold=True)

    # set background color for the first row
    fill = PatternFill(start_color='23C4ED', end_color='23C4ED', fill_type='solid')
    for row in ws.iter_rows(min_row=1, max_row=1):
        if row[0].row == 1:
            for cell in row:
                cell.fill = fill
                cell.font = font_name

    # save the workbook
    wb.save(outputFilename)
    wb.close()

#Function for automating the pivot table
def automate_pivot_table(inputFilename, outputFilename):
    # Load the Excel file into a pandas DataFrame
    # df = pd.read_excel("pivot_dataset.xlsx")
    df = pd.read_excel(inputFilename)

    # Create a pivot table
    pivot_df = pd.pivot_table(df, values='Total',
                              index='Gender', columns='Payment', aggfunc='sum')

    # Export the pivot table to an Excel file
    pivot_df.to_excel(outputFilename, sheet_name='Sheet1', index=True)

    # Create a Plotly figure
    fig = px.imshow(pivot_df)

    # Show the figure
    fig.write_image('fig.png')

In [ ]:
#passing my file named 'US_Regional_Sales_data.xlsx' as an input will create a new file named 'Salesreport.xlsx' as an output
automate_report('/content/drive/MyDrive/automation_in_python/US_Regional_Sales_Data.xlsx', '/content/drive/MyDrive/automation_in_python/SalesReport.xlsx')
#passing the 'Salesreport.xlsx' file generated using the above function will create a new formatted excel file named 'formattedReport.xlsx'
automate_formatting('/content/drive/MyDrive/automation_in_python/SalesReport.xlsx', '/content/drive/MyDrive/automation_in_python/formattedReport.xlsx')
#passing the file named 'Supermarket_sales.xlsx' will create a new file named pivotTable.xlsx
automate_pivot_table('/content/drive/MyDrive/automation_in_python/supermarket_sales.xlsx', '/content/drive/MyDrive/automation_in_python/pivotTable.xlsx')

## Live Dash app showing automated Excel **reports**

In [ ]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 26.2 MB/s eta 0:00:00


In [ ]:
from dash import Dash, html, dcc, Output, Input
import plotly.express as px
import pandas as pd
import openpyxl

df = pd.read_excel("/content/drive/MyDrive/automation_in_python/supermarket_sales.xlsx")

In [ ]:
#instantiating Dash
app = Dash(__name__)
#Defining the web layout
app.layout=html.Div([
    html.H1("Excel to Python App"),
    dcc.RadioItems(id='col-choice', options=['Gender','Customer type','City'], value='Gender'),
    dcc.Graph(id='our-graph', figure={}),
])

In [ ]:
@app.callback(
    Output('our-graph', 'figure'),
    Input('col-choice', 'value')
)
def update_graphs(column_selected):
    pivot_df = pd.pivot_table(df, values='Total', index=column_selected, columns='Payment', aggfunc='sum')
    fig = px.imshow(pivot_df)
    return fig

In [ ]:
if __name__=='__main__':
    app.run_server()

<IPython.core.display.Javascript object>